## Train model for object detection

1) usar los datos de las filtered images para entrenar una red que haga deteccion de objetos.
2) Para esto es necesario que tenga imagenes de test para validar!! asi que tengo que correr el filtering sobre las imagenes de test tambien y guardarlas, puede ser en la misma carpeta que filtered images y despues de ahi hago train, test split/
3) bueno que es lo que tengo que usar para entrenar en realidad?
   las imagenes y las anotaciones, al menos height, width, bbox, class
4) experimentos:
   *  probar con yolo o alguna de una psada
   *  hacer transfer learning
  
16 sept: voy a probar con las imagenes de tostadora en subset_images.

1) instalo yolo https://github.com/WongKinYiu/yolov7

In [1]:
import torch

### organizar imagenes para yolo

https://www.kaggle.com/code/abubakarabdulkadir/yolo-data-preprocessing

Las anotaciones tienen que estar en archivos .txt con este formato:

"\<class_id>\<x_center>\<y_center>\<width>\<height>"


In [7]:
## TODO: mover a utils

import json
import os

def convert_to_yolo_format(coco_annotation_file, output_dir):
    with open(coco_annotation_file) as f:
        data = json.load(f)

    for ann in data['annotations']:
        image_id = ann['image_id']
        img = [img for img in data['images'] if img['id'] == image_id][0]
        img_width = img['width']
        img_height = img['height']

        category_id = ann['category_id'] - 1  # Adjust class_id if needed
        bbox = ann['bbox']
        x_center = (bbox[0] + bbox[2] / 2) / img_width
        y_center = (bbox[1] + bbox[3] / 2) / img_height
        width = bbox[2] / img_width
        height = bbox[3] / img_height

        label_file = os.path.join(output_dir, f"{image_id}.txt")
        with open(label_file, 'a') as lf:
            lf.write(f"{category_id} {x_center} {y_center} {width} {height}\n")

In [8]:
coco_annotation_file = "./subset_annotations.json"

In [9]:
output_dir = "./subset_images/"

In [10]:
convert_to_yolo_format(coco_annotation_file, output_dir)

In [12]:
label_dir = 'labels/'
os.makedirs(label_dir, exist_ok=True)

In [13]:
convert_to_yolo_format(coco_annotation_file, label_dir)

### Nombre de las imagenes
el id de la anotacion no tiene los 00s que tienen las imagenes.
Armar funcioncita para eliminar los ceros de adelante. 

In [22]:
## TODO: mover a utils o transformations
import os 
def clean_image_name(directory):
    for filename in os.listdir(directory):
        if filename.endswith(".jpg"):
            file_without_ext = filename.split(".jpg")[0]
            # remove zeros from filename
            cleaned_name = str(int(file_without_ext)) + ".jpg"
            old_file = os.path.join(directory, filename)
            new_file = os.path.join(directory, cleaned_name)
            # rename files
            os.rename(old_file, new_file)

In [23]:
clean_image_name(image_dir)

### Splitting train/test

In [25]:
import os
import shutil
from sklearn.model_selection import train_test_split

image_dir = "./subset_images"
label_dir = "./labels"
train_image_dir = "./subset_images/train"
val_image_dir = "./subset_images/val"
train_label_dir = "./subset_images/train"
val_label_dir = "./subset_images/val"

os.makedirs(train_image_dir, exist_ok=True)
os.makedirs(val_image_dir, exist_ok=True)
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(val_label_dir, exist_ok=True)

images = [f for f in os.listdir(image_dir) if f.endswith('.jpg')]
labels = [f.replace('.jpg', '.txt') for f in images]

# prevent codes from failing if image/label pair doesnt exist
valid_labels = [lbl for lbl in labels if os.path.exists(os.path.join(label_dir, lbl))]
valid_images = [img for img in images if img.replace('.jpg', '.txt') in valid_labels]

# train val split
train_images, val_images, train_labels, val_labels = train_test_split(valid_images, valid_labels, test_size=0.2)

# just move files to train and val dirs
for img, lbl in zip(train_images, train_labels):
    shutil.move(os.path.join(image_dir, img), train_image_dir)
    shutil.move(os.path.join(label_dir, lbl), train_label_dir)

for img, lbl in zip(val_images, val_labels):
    shutil.move(os.path.join(image_dir, img), val_image_dir)
    shutil.move(os.path.join(label_dir, lbl), val_label_dir)
